In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 32),  
            nn.Tanh(),
            nn.Linear(32, 64), 
            nn.Tanh(),
            nn.Linear(64, 1)  
        )

    def forward(self, x, t):
        inputs = torch.cat((x, t), dim=1)
        u = self.model(inputs)
        return u

In [3]:
model = PINN()

In [4]:
def heat_loss(model, x, t, alpha):
    u = model(x, t)
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    residual = u_t - alpha * u_xx
    return torch.mean(residual ** 2)

In [5]:
def train_pinn(model, optimizer, x_train, t_train, alpha, epochs=1000):
    losses = []

    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = heat_loss(model, x_train, t_train, alpha)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.10f}")
    return losses

In [6]:
x_train = torch.rand(500, 1, requires_grad=True)
t_train = torch.rand(500, 1, requires_grad=True)

In [7]:
def plot_graph(x_train, t_train, u_pred, losses):
    plt.figure(figsize=(10, 4))
    plt.plot(losses, label="Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Epochs")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.scatter(t_train.detach().numpy(), u_pred.detach().numpy(), color='blue', label="Predicted u(x, t)")
    plt.xlabel("Time (t)")
    plt.ylabel("Temperature u(x, t)")
    plt.title("Predicted Temperature over Time")
    plt.legend()
    plt.show()

In [8]:
alpha = 0.1  

optimizer = optim.Adam(model.parameters(), lr=0.001)
losses = train_pinn(model, optimizer, x_train, t_train, alpha)

Epoch 0, Loss: 0.0147824297
Epoch 100, Loss: 0.0000117020
Epoch 200, Loss: 0.0000063522
Epoch 300, Loss: 0.0000036066
Epoch 400, Loss: 0.0000021518
Epoch 500, Loss: 0.0000013885
Epoch 600, Loss: 0.0000009409
Epoch 700, Loss: 0.0000006460
Epoch 800, Loss: 0.0000004432
Epoch 900, Loss: 0.0000003057


In [9]:
u_pred = model(x_train, t_train)

In [10]:
plot_graph()

TypeError: plot_graph() missing 4 required positional arguments: 'x_train', 't_train', 'u_pred', and 'losses'